# A/B Hypothesis Testing
# Section

* [Introduction](#section_1)
* [Data Preparation & Cleaning](#section_2)
* [Exploratory Data Analysis](#section_3)
* [Hypothesis Testing](#section_4)

<a id='section_1'></a>
## Introduction
**<u>Background information:</u>**
An e-commerce company is revamping a landing page after various analysis and research. The company want to experiment whether the new landing page will give better conversion rate before rolling out to a wider audience. 

**<u>Data:</u>**
We were given the experiment result from control and experimental/treatment group. We have hypothesis that the new page (treatment group) will give a better conversion rate.

**<u>Goal:</u>**
Help company in deciding which landing page is better (keep the old page or implement the new one). 

<a id='section_2'></a>
## Data Preparation & Cleaning
First, we need to import the data.

In [44]:
import pandas as pd
import numpy as np

In [45]:
url = 'https://docs.google.com/spreadsheets/d/1XZ6SjnbAs_bHGdznm8sUEYqmPbNlrMabIueGvpmxi2E/edit#gid=842283717'
url = url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url)
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:49,control,old_page,0
1,804228,2017-01-12 8:01:45,control,old_page,0
2,661590,2017-01-11 16:55:06,treatment,new_page,0
3,853541,2017-01-08 18:28:03,treatment,new_page,0
4,864975,2017-01-21 1:52:26,control,old_page,1


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       5000 non-null   int64 
 1   timestamp     5000 non-null   object
 2   group         5000 non-null   object
 3   landing_page  5000 non-null   object
 4   converted     5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.4+ KB


In [47]:
print('Number of missing data for each column:')
print(df.isnull().sum())
print('\nNumber of duplicated data:', df.duplicated().sum())

Number of missing data for each column:
user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

Number of duplicated data: 0


There's no missing values in the data. It also has no duplicate data. We can continue the cleaning process.

In [48]:
print(df.iloc[:,1].min(), 'to', df.iloc[:,1].max())

2017-01-02 13:42:41 to 2017-01-24 9:51:06


In [49]:
df[df.iloc[:,0].duplicated(False)]

,user_id,timestamp,group,landing_page,converted
988,698120,2017-01-22 7:09:38,control,new_page,0
1899,773192,2017-01-09 5:37:59,treatment,new_page,0
2656,698120,2017-01-15 17:13:43,control,old_page,0
2893,773192,2017-01-14 2:56:00,treatment,new_page,0


The experiment was carried out in 23 days from January 2nd 2017 to January 24th 2017. There are 2 user who viewed the landing pages twice in a different time. 

In [50]:
print('Checking typos in column group and landing_page:')
print('Column group:', df.iloc[:,2].unique())
print('Column landing_page:', df.iloc[:,3].unique())

Checking typos in column group and landing_page:
Column group: ['control' 'treatment']
Column landing_page: ['old_page' 'new_page']


No typos were found. 

Next, We should do a validation. These condition must be true for each groups.
* Condition 1: control group must receive old landing page only
* Condition 2: treatment group must receive new landing page only

To do that, I did a grouping based on those valid condition.

In [51]:
control = (df.iloc[:,2]=='control') & (df.iloc[:,3]=='old_page')
df_control = df[control].copy()
df_control.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:49,control,old_page,0
1,804228,2017-01-12 8:01:45,control,old_page,0
4,864975,2017-01-21 1:52:26,control,old_page,1
5,936923,2017-01-10 15:20:49,control,old_page,0
7,719014,2017-01-17 1:48:30,control,old_page,0


In [52]:
treatment = (df.iloc[:,2]=='treatment') & (df.iloc[:,3]=='new_page')
df_treatment = df[treatment].copy()
df_treatment.head()

,user_id,timestamp,group,landing_page,converted
2,661590,2017-01-11 16:55:06,treatment,new_page,0
3,853541,2017-01-08 18:28:03,treatment,new_page,0
6,679687,2017-01-19 3:26:47,treatment,new_page,1
8,817355,2017-01-04 17:58:09,treatment,new_page,1
9,839785,2017-01-15 18:11:07,treatment,new_page,1


<a id='section_3'></a>
## Exploratory Data Analysis (EDA)
Based on the EDA result below, the total user who joined this experiment in each group is balanced enough. The sample size is around 2500, we can use Z-test for the hypothesis testing. The conversion rate in treatment group (new landing page) is slightly higher than the conversion rate in control group (old landing page), but we can't take any conclussion about it yet, A/B hypothesis testing should be done first.

In [53]:
print('Control Group range in', df_control.iloc[:,1].min(), 'to', df_control.iloc[:,1].max())
print('Number of converted user in control group =', (df_control.iloc[:,-1]).sum())
print('Number of total user in control group =', len(df_control))
print('Conversion rate in control group =', (df_control.iloc[:,-1].mean()*100).round(2),'%')
print('Standard deviation in control group =', (df_control.iloc[:,-1].std()).round(2))
print('\nTreatment Group range in', df_treatment.iloc[:,1].min(), 'to', df_treatment.iloc[:,1].max())
print('Number of converted user in treatment group =', (df_treatment.iloc[:,-1]).sum())
print('Number of total user in treatment group =', len(df_treatment))
print('Conversion rate in treatment group:', (df_treatment.iloc[:,-1].mean()*100).round(2),'%')
print('Standard deviation in treatment group =', (df_treatment.iloc[:,-1].std()).round(2))

Control Group range in 2017-01-02 13:47:13 to 2017-01-24 9:51:06
Number of converted user in control group = 316
Number of total user in control group = 2505
Conversion rate in control group = 12.61 %
Standard deviation in control group = 0.33

Treatment Group range in 2017-01-02 13:42:41 to 2017-01-24 9:18:21
Number of converted user in treatment group = 321
Number of total user in treatment group = 2438
Conversion rate in treatment group: 13.17 %
Standard deviation in treatment group = 0.34


<a id='section_4'></a>
## Hypothesis Testing

One tailed Z-test was chosen to prove the hypothesis. Either one of these statement is true for our testing result:

**Null Hypothesis (H0)**: Conversion rate in new page **less than or equal to** the conversion rate in old page

**Alternative Hypothesis (H1)**: Conversion rate in new page **better than** the conversion rate in old page

In [54]:
from statsmodels.stats.weightstats import ztest
(stat, pvalue) = ztest(df_treatment.iloc[:,-1], df_control.iloc[:,-1], alternative='larger')
print('Z-score =', stat)
print('p-value =', pvalue)
if (pvalue<0.05):
    print('The p=value is less than 0.05, we have enough evidence to reject null hypothesis')
else:
    print('The p=value is more than 0.05, we do not have enough evidence to reject null hypothesis')

Z-score = 0.5787432513872296
p-value = 0.2813812136899914
The p=value is more than 0.05, we do not have enough evidence to reject null hypothesis


**<u>Result</u>**:
We don't have enough evidence to reject null hypothesis, thus we **accept null hypothesis.** New landing page doesn't give better conversion rate than the old one. 

## Recommendation
The company can keep using the old landing page, because the hypothesis testing result show that conversion rate in new landing page doesn't sginificantly differ with the one in old landing page 